In [120]:
import time
import urllib.parse as urlparser
import uuid
from io import BytesIO
from os import path
from lxml import etree
from urllib.request import Request, urlopen
import requests
from PIL import Image
import six
import shutil
import os.path as pth

In [94]:
Pages = ["https://multi.xnxx.com/category/non-nude/", "http://viewgals.com/non-nude/"]

In [ ]:
Pages = ["https://multi.xnxx.com/category/non-nude/p-2/", "https://multi.xnxx.com/category/non-nude/p-3/", "https://multi.xnxx.com/category/non-nude/p-4/"

In [95]:
def get_text(element):
    text = get_attr(element, "title")
    text += get_attr(element.getparent(), "title")
    return text + "".join(element.getparent().itertext()).replace("\n", " ")


def get_attr(node, attr):
    if node is not None:
        if node.get(attr) is not None:
            return node.get(attr)
    return ""


def get_format(uri):
    uri_path = urlparser.urlparse(uri).path
    _, ext = path.splitext(uri_path)
    if ext.lower() in EXT:
        return ext.lower()
    return ""

In [96]:
parser = etree.HTMLParser(remove_comments=True, default_doctype=False, recover=True)

EXT = [".jpg", ".jpeg", ".gif", ".png"]

def find_img_data(html):
    try:
        tree = etree.parse(BytesIO(html), parser)
    except Exception as e:
        raise Exception("Cannot parse images : " + e)
    imgs_list = tree.findall("//img")
    links = []
    for img in imgs_list:
        l = ""
        if "src" in img.attrib:
            l = img.attrib["src"]
        elif "data-src" in img.attrib:
            l = img.attrib["data-src"]
        if l[:4] == "http":
            links.append(l)
    return links


In [97]:
def scale(content, min_dim):
    """ Aspect-ratio preserving scale such that the smallest dim is equal to `min_dim` """

    image = Image.open(content)

    # no scaling, keep images full size
    if min_dim == -1:
        return image

    # aspect-ratio preserving scale so that the smallest dimension is `min_dim`
    width, height = image.size
    scale_dimension = width if width < height else height
    scale_ratio = float(min_dim) / scale_dimension

    if scale_ratio == 1:
        return image

    return image.resize(
        (int(width * scale_ratio), int(height * scale_ratio)),
        Image.ANTIALIAS,
    )

In [112]:
def read_image(response, min_dim=300):
    """ Download response in chunks and convert to a scaled Image object """

    content = six.BytesIO()
    shutil.copyfileobj(response.raw, content)
    content.seek(0)
    
    if min_dim > 0 :
        return scale(content, min_dim)
    else:
        return Image.open(content)

In [113]:
SAVING_PATH="/data/porn/data/train/sexy/"

In [126]:
def get_qwant_data(text, save_dir, nb_images=100):
    try:
        r = requests.get("https://api.qwant.com/api/search/images",
            params={
                'count': nb_images,
                'q': text,
                't': 'images',
                'safesearch': 1,
                'locale': 'fr_FR',
                'uiv': 4
            },
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
            },
            stream=True, timeout=2.0
        )
    except Exception as e:
        print(e)
        return
    try:
        response = r.json().get('data').get('result').get('items')
    except Exception as e:
        print(e)
        return
    urls = [r.get('media') for r in response]
    for i, url in enumerate(urls):
        print(f"{i}/{len(urls)}", end="\r")
        id = str(uuid.uuid5(uuid.NAMESPACE_URL, url)) + '.jpg'
        if 'base64,' in url:
            image_data = url.split(',')[1]
            buffered = BytesIO()
            im = Image.open(buffered, format="JPEG").convert("RGB")
            im.save(pth.join(save_dir,id))
        else:
            try:
                response = requests.get(url,stream=True, timeout=2.0)
            except Exception as e:
                print(e)
                continue
            if response.status_code < 400:
                try :
                    im = read_image(response).convert("RGB")
                    im.save(pth.join(save_dir,id))
                except Exception as e:
                    print(e)
    return

In [127]:
QUERIES= ["nude", "woman beach", "waifu"]

In [128]:
for q in QUERIES:
    get_qwant_data(q, "/data/porn/tmp/")

cannot identify image file <_io.BytesIO object at 0x7fc064774a70>
HTTPSConnectionPool(host='wallpapersite.com', port=443): Read timed out. (read timeout=2.0)
cannot identify image file <_io.BytesIO object at 0x7fc0646ee350>
cannot identify image file <_io.BytesIO object at 0x7fc0646ddc50>


In [100]:
for page in Pages:
    data = Request(page)
    html = urlopen(data).read()
    links = find_img_data(html)
    for i, link in enumerate(links):
        print(f"{i}/{len(links)}", end='\r')
        try:
            response = requests.get(link, stream=True, timeout=2.0)
        except:
            continue
        image = read_image(response, 256)
        image.save(f"{SAVING_PATH}{uuid.uuid5(uuid.NAMESPACE_URL, link)}.jpg")

OSError: cannot write mode P as JPEG